# Ansible Note | chusiang

This is a note for Ansible with jupyter (iPython Notebook), [fork me on GitHub](https://github.com/chusiang/sandbox/tree/develop/ansible).

## Environment

In [8]:
!whoami

root


In [2]:
!pwd

/vagrant


In [3]:
!cat hosts

# syntax: servername  options
# 常用選項
# ansible_ssh_host -- Remote Host IP
# ansible_ssh_user -- Remote SSH User Name
# ansible_ssh_private_key_file -- SSH Key
# ansible_ssh_port -- ssh port
# ansible_ssh_pass -- ssh password

# 如果有定義到 ansible.cfg 的 [ defaults ] 就可以不列出
# testserver   ansible_ssh_host=xxx.xxx.xxx.xxx ansible_ssh_port=xx ansible_ssh_pass

ubuntu1   ansible_ssh_host=33.33.33.31  ansible_ssh_port=22

# docker1       ansible_ssh_host=172.17.0.3 ansible_ssh_port=22

# 針對不同的平台可能 key 以及使用者不一樣可以額外指定

# 也可以用群組的方式
[vagrant]
ubuntu1

[test]
ubuntu1

# [docker]


## Ad-Hoc Commands

相對於 Ad-Hoc 和 ansible-playbook，我們可以用 command line 和 shell script 來理解。  

* [Introduction To Ad-Hoc Commands | Ansible Documentation](http://docs.ansible.com/ansible/intro_adhoc.html)
* [Introduction To Ad-Hoc Commands | Ansible 中文權威指南 1.0.1](http://chusiang.github.io/ansible-docs-translate/intro_adhoc.html)

### Command

因 command 這模組可省略 `-m command`，故簡寫之。

In [37]:
!ansible all -a 'whoami'

ubuntu1 | SUCCESS | rc=0 >>
vagrant



In [29]:
!ansible all -m command -a 'lsb_release -a'

ubuntu1 | SUCCESS | rc=0 >>
Distributor ID:	Ubuntu
Description:	Ubuntu 14.04.4 LTS
Release:	14.04
Codename:	trustyNo LSB modules are available.



### Ping

In [7]:
!ansible all -m ping

ubuntu1 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


### File Transfer

* Copy file.

In [4]:
!ansible all -m copy -a "src=README.md dest=/tmp/"

ubuntu1 | SUCCESS => {
    "changed": true, 
    "checksum": "8cdab64639ae175a2e4dfcac34fa42afb93ea8d3", 
    "dest": "/tmp/README.md", 
    "gid": 1000, 
    "group": "vagrant", 
    "md5sum": "3eef0c4dbb0f8462a5df3823ff40fd17", 
    "mode": "0664", 
    "owner": "vagrant", 
    "size": 1158, 
    "src": "/home/vagrant/.ansible/tmp/ansible-tmp-1457325827.21-156048055407184/source", 
    "state": "file", 
    "uid": 1000
}


* Change file permssion, like this:

        $ chmod 600 /tmp/REEADME.md

In [10]:
!ansible all -m file -a "dest=/tmp/README.md mode=600"

ubuntu1 | SUCCESS => {
    "changed": true, 
    "gid": 1000, 
    "group": "vagrant", 
    "mode": "0600", 
    "owner": "vagrant", 
    "path": "/tmp/README.md", 
    "size": 1158, 
    "state": "file", 
    "uid": 1000
}


In [13]:
!ansible all -m command -a "ls -l /tmp"

ubuntu1 | SUCCESS | rc=0 >>
total 4
-rw------- 1 vagrant vagrant 1158 Mar  7 04:43 README.md



* Change file permssion, like this:

        $ sudo chmod 600 /tmp/REEADME.md && sudo chown root:root /tmp/README.md

In [14]:
!ansible all -m file -a "dest=/tmp/README.md mode=600 owner=root group=root" -s

ubuntu1 | SUCCESS => {
    "changed": true, 
    "gid": 0, 
    "group": "root", 
    "mode": "0600", 
    "owner": "root", 
    "path": "/tmp/README.md", 
    "size": 1158, 
    "state": "file", 
    "uid": 0
}


In [15]:
!ansible all -m command -a "ls -l /tmp"

ubuntu1 | SUCCESS | rc=0 >>
total 4
-rw------- 1 root root 1158 Mar  7 04:43 README.md



* Remove file.

        `sudo rm /tmp/REEADME.md`

In [16]:
!ansible all -m file -a "dest=/tmp/README.md state=absent" -s

ubuntu1 | SUCCESS => {
    "changed": true, 
    "path": "/tmp/README.md", 
    "state": "absent"
}


In [17]:
!ansible all -m command -a "ls -l /tmp"

ubuntu1 | SUCCESS | rc=0 >>
total 0



### Package Manage

It's like

    $ sudo apt-get update && sudo apt-get install nginx

In [53]:
!ansible all -m apt -a "name=nginx state=latest" -s

ubuntu1 | SUCCESS => {
    "cache_update_time": 0, 
    "cache_updated": false, 
    "changed": false
}


### Users and Groups

It's like `adduser`, `deluser`...

* Add **foo** user.

In [18]:
!ansible all -m user -a "name=foo password=FooPasswordd" -s

ubuntu1 | SUCCESS => {
    "changed": true, 
    "comment": "", 
    "createhome": true, 
    "group": 1002, 
    "home": "/home/foo", 
    "name": "foo", 
    "password": "NOT_LOGGING_PASSWORD", 
    "shell": "", 
    "state": "present", 
    "stderr": "useradd: warning: the home directory already exists.\nNot copying any file from skel directory into it.\n", 
    "system": false, 
    "uid": 1002
}


In [19]:
!ansible all -m shell -a "tail -n 3 /etc/group"

ubuntu1 | SUCCESS | rc=0 >>
ubuntu:x:1001:
utempter:x:115:
foo:x:1002:



* Delete **foo** user.

In [20]:
!ansible all -m user -a "name=foo state=absent" -s

ubuntu1 | SUCCESS => {
    "changed": true, 
    "force": false, 
    "name": "foo", 
    "remove": false, 
    "state": "absent"
}


In [21]:
!ansible all -m shell -a "tail -n 3 /etc/group"

ubuntu1 | SUCCESS | rc=0 >>
puppet:x:114:
ubuntu:x:1001:
utempter:x:115:



### Deploying From Source Control

使用 Git 來部署 my dotfiles。

In [39]:
!ansible all -m git -a "repo=https://github.com/chusiang/tuxENV.git dest=~/vcs/tuxENV version=HEAD"

ubuntu1 | SUCCESS => {
    "after": "e8738ba3ebea96ed7e7c582c291d9bf8d683e172", 
    "before": null, 
    "changed": true
}


In [47]:
!ansible all -m shell -a "cd ~/vcs/tuxENV/ && git branch -v"

ubuntu1 | SUCCESS | rc=0 >>
* master e8738ba Merge branch 'develop'



In [46]:
!ansible all -m shell -a "cd ~/vcs/tuxENV/ && make install"

ubuntu1 | SUCCESS | rc=0 >>
=> Install start.
cat home/_bash_aliases > /home/vagrant/.bash_aliases
cat home/_bash_export > /home/vagrant/.bash_export
cat home/_bash_profile > /home/vagrant/.bash_profile
cat home/_bashrc > /home/vagrant/.bashrc
cat home/_screenrc > /home/vagrant/.screenrc
cat home/_tmux.conf > /home/vagrant/.tmux.conf
cat home/_inputrc > /home/vagrant/.inputrc
cat home/_gitconfig > /home/vagrant/.gitconfig
cat home/_myclirc > /home/vagrant/.myclirc
cp -a home/bin/ /home/vagrant/bin 
=> Done.



### Managing Services

* start nginx.

In [58]:
!ansible all -m service -a "name=nginx state=started" -s

ubuntu1 | SUCCESS => {
    "changed": false, 
    "name": "nginx", 
    "state": "started"
}


* restart nginx.

In [61]:
!ansible all -m service -a "name=nginx state=restarted" -s

ubuntu1 | SUCCESS => {
    "changed": true, 
    "name": "nginx", 
    "state": "started"
}


* stop nginx.

In [60]:
!ansible all -m service -a "name=nginx state=stopped" -s

ubuntu1 | SUCCESS => {
    "changed": true, 
    "name": "nginx", 
    "state": "stopped"
}


## Playbooks

* [Playbooks | Ansible Documentation](http://docs.ansible.com/ansible/playbooks.html)
* [Playbooks | Ansible 中文權威指南 1.0.1](http://chusiang.github.io/ansible-docs-translate/playbooks.html)


In [3]:
!cat upgrade-openssl.yml

#!/usr/bin/env ansible-playbook
# ============================================================
#  Author: chusiang / chusiang.lai (at) gmail.com
#  Filename: upgrade-openssl.yml
#  Modified: 2016-03-03 11:44
#  Description: security upgrade the openssl package
#  Reference: 
#
#   1. ansible print debug msg variable - Server Fault
#    - http://serverfault.com/questions/695786/ansible-print-debug-msg-variable
#
# =========================================================== 

---
- name: upgrade package of openssl 
  hosts: all
  sudo: True
  tasks:
    - name: upgrade on ubuntu
      apt: name=openssl update_cache=yes state=latest
      when: ansible_distribution == "Ubuntu"

    - name: check version on ubuntu
      command: dpkg-query -W openssl
      register: dpkg_package_version
      when: ansible_distribution == "Ubuntu"

    - name: print version
      debug:
        msg: "OpenSSL Version: {{ dpkg_package_version.stdout }}"
      when: ansible_distribution == "Ubuntu"

    - nam

In [7]:
!ansible-playbook upgrade-openssl.yml

[DEPRECATION WARNING]: Instead of sudo/sudo_user, use become/become_user and 
make sure become_method is 'sudo' (default). This feature will be removed in a 
future release. Deprecation warnings can be disabled by setting 
deprecation_warnings=False in ansible.cfg.

PLAY [upgrade package of openssl] **********************************************

TASK [setup] *******************************************************************
ok: [ubuntu1]

TASK [upgrade on ubuntu] *******************************************************
ok: [ubuntu1]

TASK [check version on ubuntu] *************************************************
changed: [ubuntu1]

TASK [print version] ***********************************************************
ok: [ubuntu1] => {
    "msg": "OpenSSL Version: openssl\t1.0.1f-1ubuntu2.18"
}

TASK [upgrade on centos] *******************************************************
skipping: [ubuntu1]

TASK [check version on centos] *************************************************
skipping: [ubu